# Part 1

Implement FROM SCRATCH a tokenizer based on the BytePair encoding algorithm (link). You
are only allowed to use standard Python libraries and objects (lists, arrays, dictionaries, and
collections library). Use of existing frameworks (such as nltk, HuggingFace, Spacy, TextBlob) is
not allowed.
</n>
<br>
Specifically, you are required to create a Tokenizer class in python, which implements the
following methods:
* learn_vocablury(), which takes as parameter the corpus number of merges and learns
the split rules and frequencies; and </n>
* tokenize(), which takes as input a sample and tokenizes it based on the learnt rules.

In [6]:
from collections import Counter, defaultdict

def get_vocab(text):
    vocab = Counter(text.split())
    return {' '.join(word): freq for word, freq in vocab.items()}

def get_stats(vocab):
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[symbols[i], symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, vocab):
    new_vocab = {}
    bigram = ' '.join(pair)
    replacement = ''.join(pair)
    for word in vocab:
        new_word = word.replace(bigram, replacement)
        new_vocab[new_word] = vocab[word]
    return new_vocab

def tokenize_samples(samples, vocab):
    tokens_per_sample = []
    for sample in samples:
        tokens = sample.split()
        for i in range(len(tokens)):
            if i < len(tokens) - 1:
                pair = (tokens[i], tokens[i+1])
                if pair in vocab:
                    tokens[i] = ''.join(pair) + '$'
                    tokens[i+1] = ''
        tokens_per_sample.append([token for token in tokens if token]) 
    return tokens_per_sample

# Read from corpus.txt
with open("corpus.txt", "r") as corpus_file:
    text = corpus_file.read()

vocab = get_vocab(text)
# print("Initial vocabulary:", vocab)
num_merges = 5  

for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best_pair = max(pairs, key=pairs.get)
    vocab = merge_vocab(best_pair, vocab)

# Save results back to corpus.txt
with open("corpus.txt", "w") as corpus_file:
    corpus_file.write(text)  # Writing the original text
    for token in vocab.keys():
        corpus_file.write('\n' + ' '.join(list(token.replace(' ', '$'))))  # Writing tokens with $ separator

# Save all possible tokens in the vocabulary to tokens.txt
with open("tokens.txt", "w") as tokens_file:
    for token in vocab.keys():
        tokens = list(token.replace(' ', '$'))
        for i in range(len(tokens)):
            for j in range(i, len(tokens)):
                tokens_file.write(''.join(tokens[i:j+1]) + '\n')


# Save all the merge rules to merge_rules.txt
with open("merge_rules.txt", "w") as rules_file:
    for rule in vocab.keys():
        rules_file.write(rule.replace(' ', ',') + '\n')

# Tokenize a set of test samples and write the tokens to tokenized_samples.txt
test_samples = ["low widest", "newest lower"]
tokenized_samples = tokenize_samples(test_samples, vocab)

with open("tokenized_samples.txt", "w") as sample_file:
    for tokens in tokenized_samples:
        sample_file.write(','.join(tokens) + '\n')

# Part 2

In [ ]:
class BigramLM():
    def __init__(self, tokens):
        self.tokens = tokens
        self.bigram_counts = self.get_bigram_counts()
        self.unigram_counts = self.get_unigram_counts()
        self.bigram_probs = self.get_bigram_probs()
        self.unigram_probs = self.get_unigram_probs()
        self.bigram_laplace_probs = self.get_bigram_laplace_probs()
        self.unigram_laplace_probs = self.get_unigram_laplace_probs()

    def get_bigram_counts(self):
        bigram_counts = defaultdict(int)
        for i in range(len(self.tokens) - 1):
            bigram_counts[self.tokens[i], self.tokens[i+1]] += 1
        return bigram_counts

    def get_unigram_counts(self):
        unigram_counts = defaultdict(int)
        for token in self.tokens:
            unigram_counts[token] += 1
        return unigram_counts

    def get_bigram_probs(self):
        bigram_probs = defaultdict(int)
        for bigram, count in self.bigram_counts.items():
            bigram_probs[bigram] = count / self.unigram_counts[bigram[0]]
        return bigram_probs

    def get_unigram_probs(self):
        unigram_probs = defaultdict(int)
        for unigram, count in self.unigram_counts.items():
            unigram_probs[unigram] = count / len(self.tokens)
        return unigram_probs

    def get_bigram_laplace_probs(self):
        bigram_laplace_probs = defaultdict(int)
        for bigram, count in self.bigram_counts.items():
            bigram_laplace_probs[bigram] = (count + 1) / (self.unigram_counts[bigram[0]] + len(self.unigram_counts))
        return bigram_laplace_probs

    def get_unigram_laplace_probs(self):
        unigram_laplace_probs = defaultdict(int)
        for unigram, count in self.unigram_counts.items():
            unigram_laplace_probs[unigram] = (count + 1) / (len(self.tokens) + len(self.unigram_counts))
        return unigram_laplace_probs
    
    def get_bigram_interpolated_probs(self):
        bigram_interpolated_probs = defaultdict(int)
        for bigram, count in self.bigram_counts.items():
            bigram_interpolated_probs[bigram] = 0.5 * (count / self.unigram_counts[bigram[0]]) + 0.5 * (self.unigram_counts[bigram[1]] / len(self.tokens))
        return bigram_interpolated_probs
